<img src="images/translation.jpg" width="800">

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Dataset-load" data-toc-modified-id="Dataset-load-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset load</a></span></li><li><span><a href="#Model-Architecture" data-toc-modified-id="Model-Architecture-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Model Architecture</a></span><ul class="toc-item"><li><span><a href="#Encoder" data-toc-modified-id="Encoder-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Encoder</a></span></li><li><span><a href="#Decoder" data-toc-modified-id="Decoder-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Decoder</a></span></li><li><span><a href="#Train-Model" data-toc-modified-id="Train-Model-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Train Model</a></span><ul class="toc-item"><li><span><a href="#Save-the-weights" data-toc-modified-id="Save-the-weights-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Save the weights</a></span></li><li><span><a href="#Load-the-weights,-if-you-close-the-application" data-toc-modified-id="Load-the-weights,-if-you-close-the-application-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Load the weights, if you close the application</a></span></li><li><span><a href="#Inference-Setup" data-toc-modified-id="Inference-Setup-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Inference Setup</a></span></li><li><span><a href="#Decode-sample-sequeces" data-toc-modified-id="Decode-sample-sequeces-3.3.4"><span class="toc-item-num">3.3.4&nbsp;&nbsp;</span>Decode sample sequeces</a></span></li></ul></li></ul></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#Train-Dataset" data-toc-modified-id="Train-Dataset-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Train Dataset</a></span></li><li><span><a href="#Validation-Dataset" data-toc-modified-id="Validation-Dataset-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Validation Dataset</a></span></li><li><span><a href="#Evaluating-with-new-text:" data-toc-modified-id="Evaluating-with-new-text:-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Evaluating with new text:</a></span></li></ul></li></ul></div>

# Imports

In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import re

from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


# Dataset load
http://www.manythings.org/anki/  (Download and unzip spa-eng.zip file)

In [3]:
lines= pd.read_table('data/Translator/spa.txt', names=['eng', 'spa','other'])

In [4]:
lines.shape

(122936, 3)

In [5]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.spa=lines.spa.apply(lambda x: x.lower())

In [6]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.spa=lines.spa.apply(lambda x: re.sub("'", '', x))

In [7]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.spa=lines.spa.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [8]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))

In [9]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.spa=lines.spa.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.spa=lines.spa.apply(lambda x: re.sub(" +", " ", x))

In [10]:
# Add start and end tokens to target sequences
lines.spa = lines.spa.apply(lambda x : 'START_ '+ x + ' _END')

In [11]:
lines.sample(10)

,eng,spa,other
45429,he likes to live in tokyo,START_ le gusta vivir en tokio _END,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
40971,he insulted me in public,START_ él me insultó en público _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
88752,ive drunk way too much coffee today,START_ he bebido demasiado café hoy _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
23275,who are those women,START_ ¿quiénes son esas mujeres _END,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
86239,she died two days after his arrival,START_ ella murió dos días después de su llega...,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4080,dont be evil,START_ no seáis malas _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
113578,please remove your shoes before entering the h...,START_ por favor quítese los zapatos antes de ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
61041,tom is honest so i like him,START_ tom es honesto por eso me gusta _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
83826,tom couldnt say anything publicly,START_ tom no podía decir nada en público _END,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
88843,it shouldnt take me long to do that,START_ no debería llevarme mucho tiempo hacerl...,CC-BY 2.0 (France) Attribution: tatoeba.org #6...


In [12]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of French 
all_spa_words=set()
for spa in lines.spa:
    for word in spa.split():
        if word not in all_spa_words:
            all_spa_words.add(word)

In [13]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

47

In [14]:
# Max Length of target sequence
lenght_list=[]
for l in lines.spa:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

55

In [15]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_spa_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_spa_words)
num_encoder_tokens, num_decoder_tokens

(13467, 27431)

In [16]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

27432

In [17]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [18]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [19]:
#lines = shuffle(lines)
lines.head(10)

,eng,spa,other
0,go,START_ ve _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,go,START_ vete _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,go,START_ vaya _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,go,START_ váyase _END,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
4,hi,START_ hola _END,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5,run,START_ ¡corre _END,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
6,run,START_ ¡corran _END,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
7,run,START_ ¡corra _END,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
8,run,START_ ¡corred _END,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
9,run,START_ corred _END,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [20]:
# Train - Test Split
from sklearn.model_selection import train_test_split

X, y = lines.eng, lines.spa
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((110642,), (12294,))

In [21]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Model Architecture 
## Encoder

In [22]:
latent_dim = 50

In [23]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


## Decoder

In [24]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [25]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [26]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

## Train Model

In [56]:
#model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
#                    steps_per_epoch = train_samples/batch_size,
#                    epochs=epochs,
#                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
#                    validation_steps = val_samples/batch_size)

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
493/492 [==============================] - 189s 384ms/step - loss: 1.9431 - acc: 0.1953 - val_loss: 1.1522 - val_acc: 0.2284
Epoch 2/50
493/492 [==============================] - 188s 382ms/step - loss: 1.6869 - acc: 0.2455 - val_loss: 1.0683 - val_acc: 0.2665
Epoch 3/50
493/492 [==============================] - 188s 381ms/step - loss: 1.5581 - acc: 0.2922 - val_loss: 0.9886 - val_acc: 0.3174
Epoch 4/50
493/492 [==============================] - 188s 381ms/step - loss: 1.4486 - acc: 0.3302 - val_loss: 0.9258 - val_acc: 0.3457
Epoch 5/50
493/492 [==============================] - 188s 381ms/step - loss: 1.3682 - acc: 0.3576 - val_loss: 0.8843 - val_acc: 0.3691
Epoch 6/50
493/492 [==============================] - 188s 382ms/step - loss: 1.3036 - acc: 0.3809 - val_loss: 0.8497 - val_acc: 0.3916
Epoch 7/50
493/492 [==============================] - 188s 382ms/step - loss: 1.2504 - acc: 0.4027 - val_loss: 0.8185 - val_acc: 0.4091
Epoch 8/50
493/492 [============================

### Save the weights

In [58]:
#model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [35]:
model.load_weights('data/Translator/nmt_weights_complete_eng_esp.h5')

### Inference Setup

In [36]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs2] + decoder_states2)

### Decode sample sequeces

In [37]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
       
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# Evaluation 
## Train Dataset

In [38]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1
for k in range(10):
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('\nInput English sentence:', X_train[k:k+1].values[0])
    print('Actual Spanish Translation:', y_train[k:k+1].values[0][6:-4])
    print('Predicted Spanish Translation:', decoded_sentence[:-4])


Input English sentence: the baby needs its mother
Actual Spanish Translation:  el bebé necesita a su madre 
Predicted Spanish Translation:  el bebé necesita que él está 

Input English sentence: you lost
Actual Spanish Translation:  has perdido 
Predicted Spanish Translation:  tú te vayas 

Input English sentence: i am married
Actual Spanish Translation:  estoy casada 
Predicted Spanish Translation:  estoy casado 

Input English sentence: it wasnt tom who left the water running
Actual Spanish Translation:  no fue tom quien dejó la llave del agua abierta 
Predicted Spanish Translation:  no fue la gente que de ver a su madre estaba brome

Input English sentence: i am happy to see you again
Actual Spanish Translation:  estoy feliz de verte otra vez 
Predicted Spanish Translation:  estoy contento de verte de nuevo 

Input English sentence: tom was really glad to hear that mary hadnt been injured in the accident
Actual Spanish Translation:  tom se alegró mucho de saber que mary no había si

## Validation Dataset

In [39]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1
for k in range(10):
    (input_seq, actual_output), _ = next(val_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('\nInput English sentence:', X_test[k:k+1].values[0])
    print('Actual Spanish Translation:', y_test[k:k+1].values[0][6:-4])
    print('Predicted Spanish Translation:', decoded_sentence[:-4])


Input English sentence: it was the perfect moment for a kiss
Actual Spanish Translation:  fue el momento perfecto para un beso 
Predicted Spanish Translation:  fue el trabajo allí es fácil de un accidente 

Input English sentence: he had a lot to do with that project
Actual Spanish Translation:  él tuvo mucho que ver con ese proyecto 
Predicted Spanish Translation:  él tuvo que hacer mucho tiempo en avión 

Input English sentence: i think i have a theory about that
Actual Spanish Translation:  creo que tengo una teoría sobre eso 
Predicted Spanish Translation:  creo que tengo una reunión es un 

Input English sentence: they took him to the hospital for his fever
Actual Spanish Translation:  ellos lo llevaron al hospital por su fiebre 
Predicted Spanish Translation:  ellos se lo hizo un trabajo a la cocina 

Input English sentence: i need you to help me
Actual Spanish Translation:  necesito que me ayudes 
Predicted Spanish Translation:  necesito que me te necesito 

Input English sente

## Evaluating with new text:

In [40]:
inputText = ['who is your father']
outputText = ['quien es tu padre']

train_gen = generate_batch(inputText, outputText, batch_size=1)
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)

print('Input English sentence:', inputText[0])
print('Actual Spanish Translation:', outputText[0])
print('Predicted Spanish Translation:', decoded_sentence[:-4])

Input English sentence: who is your father
Actual Spanish Translation: quien es tu padre
Predicted Spanish Translation:  ¿quién es tu padre 
